# Probabilistic Weather Forecasting (ERA5 — Dublin)
This notebook follows the lecture on **probabilistic forecasting with neural networks**.
Using `era5_ireland3_t2m_wind_2024.csv`, we predict **next-hour Dublin 2m temperature** from the **previous 24 hours**.

**Plan:**
1) Scoring rules (Gaussian NLL, CRPS) + calibration (PIT, reliability).
2) Baseline probabilistic model.
3) Probabilistic **1D CNN** → (mu, sigma).
4) Probabilistic **LSTM** → (mu, sigma).
5) Diagnostics & optional quantile head.

## 0) Setup

In [ ]:
# %pip install pandas numpy scikit-learn torch matplotlib
import warnings, pathlib
warnings.filterwarnings('ignore')
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
CSV_PATH='era5_ireland3_t2m_wind_2024.csv'
assert pathlib.Path(CSV_PATH).exists(), 'Place the CSV next to this notebook.'
device='cpu'; print('Device:', device)

## 1) Load, split, windows (24→next)

In [ ]:
df=pd.read_csv(CSV_PATH, parse_dates=['time']).sort_values('time').reset_index(drop=True)
series=df[['Dublin_t2m_degC']].copy()
n=len(df); i_tr=int(0.70*n); i_va=int(0.85*n)
sc=StandardScaler().fit(series.iloc[:i_tr])
x_all=sc.transform(series.values)
T=24
def make_windows(x,T=24):
    Xs,ys=[],[]
    for t in range(len(x)-T):
        Xs.append(x[t:t+T]); ys.append(x[t+T])
    return np.stack(Xs), np.stack(ys)
X_all,y_all=make_windows(x_all,T)
M=len(X_all); i_tr_w=int(0.70*M); i_va_w=int(0.85*M)
Xtr,ytr=X_all[:i_tr_w],y_all[:i_tr_w]
Xva,yva=X_all[i_tr_w:i_va_w],y_all[i_tr_w:i_va_w]
Xte,yte=X_all[i_va_w:],y_all[i_va_w:]
class SeqDataset(Dataset):
    def __init__(self,X,y):
        self.X=torch.tensor(X,dtype=torch.float32); self.y=torch.tensor(y,dtype=torch.float32)
    def __len__(self): return len(self.X)
    def __getitem__(self,i): return self.X[i],self.y[i]
tr_dl=DataLoader(SeqDataset(Xtr,ytr),batch_size=128,shuffle=True)
va_dl=DataLoader(SeqDataset(Xva,yva),batch_size=256)
te_dl=DataLoader(SeqDataset(Xte,yte),batch_size=256)
print('Windows:',Xtr.shape,ytr.shape,'|',Xva.shape,yva.shape,'|',Xte.shape,yte.shape)

## 2) Scoring rules & utilities

In [ ]:
import math
def gaussian_nll(mu,sigma,y):
    var=sigma**2
    return 0.5*torch.log(2*torch.pi*var)+0.5*((y-mu)**2)/var
def crps_gaussian(mu,sigma,y):
    eps=1e-6; sigma=torch.clamp(sigma,min=eps)
    z=(y-mu)/sigma
    Phi=0.5*(1+torch.erf(z/np.sqrt(2.0)))
    phi=torch.exp(-0.5*z**2)/np.sqrt(2*np.pi)
    return sigma*( z*(2*Phi-1)+2*phi-1/np.sqrt(np.pi) )
def to_celsius(pred_s,true_s,scaler):
    return scaler.inverse_transform(pred_s), scaler.inverse_transform(true_s)

## 3) Baseline probabilistic model

In [ ]:
with torch.no_grad():
    mu_tr=torch.tensor(Xtr[:,-1,:]); err= torch.tensor(ytr)-mu_tr
    sigma_hat=float(torch.sqrt((err**2).mean()))
def eval_baseline(X,y,scaler):
    mu=torch.tensor(X[:,-1,:]); sigma=torch.full_like(mu,fill_value=sigma_hat)
    nll=gaussian_nll(mu,sigma,torch.tensor(y)).mean().item()
    crps=crps_gaussian(mu,sigma,torch.tensor(y)).mean().item()
    mu_c,y_c=scaler.inverse_transform(mu.numpy()), scaler.inverse_transform(y)
    rmse=float(np.sqrt(((mu_c-y_c)**2).mean()))
    return nll,crps,rmse
print('sigma_hat (scaled):',round(sigma_hat,4))
print('Baseline te metrics:', eval_baseline(Xte,yte,sc))

## 4) Probabilistic 1D CNN (mu, sigma) with Gaussian NLL

In [ ]:
class ProbCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv1d(1,8,3,padding=1)
        self.conv2=nn.Conv1d(8,16,3,padding=1)
        self.pool=nn.AdaptiveAvgPool1d(1)
        self.head=nn.Linear(16,2)
        self.softplus=nn.Softplus()
    def forward(self,x):
        x=x.transpose(1,2)
        h=torch.relu(self.conv1(x)); h=torch.relu(self.conv2(h))
        h=self.pool(h).squeeze(-1)
        out=self.head(h)
        mu=out[:,:1]; sigma=self.softplus(out[:,1:])+1e-3
        return mu,sigma
torch.manual_seed(0)
model_cnn=ProbCNN(); opt=torch.optim.Adam(model_cnn.parameters(),lr=3e-3)
def eval_nll(model,loader):
    model.eval(); tot=0;n=0
    with torch.no_grad():
        for xb,yb in loader:
            mu,sig=model(xb)
            l=gaussian_nll(mu,sig,yb).mean().item(); tot+=l*len(xb); n+=len(xb)
    return tot/n
for ep in range(10):
    model_cnn.train(); run=0;ntr=0
    for xb,yb in tr_dl:
        opt.zero_grad(); mu,sig=model_cnn(xb)
        loss=gaussian_nll(mu,sig,yb).mean(); loss.backward(); opt.step()
        run+=loss.item()*len(xb); ntr+=len(xb)
    val=eval_nll(model_cnn,va_dl)
    print(f'Epoch {ep+1:02d} | train NLL={run/ntr:.4f} | val NLL={val:.4f}')
# Test
model_cnn.eval(); P_mu=[];P_sig=[];Y=[]
with torch.no_grad():
    for xb,yb in te_dl:
        mu,sig=model_cnn(xb); P_mu.append(mu.numpy()); P_sig.append(sig.numpy()); Y.append(yb.numpy())
P_mu=np.vstack(P_mu); P_sig=np.vstack(P_sig); Y=np.vstack(Y)
nll_te=float(gaussian_nll(torch.tensor(P_mu),torch.tensor(P_sig),torch.tensor(Y)).mean())
crps_te=float(crps_gaussian(torch.tensor(P_mu),torch.tensor(P_sig),torch.tensor(Y)).mean())
mu_c, y_c = sc.inverse_transform(P_mu), sc.inverse_transform(Y)
rmse_mu=float(np.sqrt(((mu_c-y_c)**2).mean()))
print('CNN te — NLL:',round(nll_te,3),'CRPS:',round(crps_te,3),'RMSE °C:',round(rmse_mu,3))

## 5) Probabilistic LSTM (mu, sigma)

In [ ]:
class ProbLSTM(nn.Module):
    def __init__(self,hidden=32):
        super().__init__(); self.lstm=nn.LSTM(1,hidden,batch_first=True); self.head=nn.Linear(hidden,2); self.softplus=nn.Softplus()
    def forward(self,x):
        o,_=self.lstm(x); h=o[:,-1,:]; out=self.head(h); mu=out[:,:1]; sigma=self.softplus(out[:,1:])+1e-3; return mu,sigma
model_lstm=ProbLSTM(); opt=torch.optim.Adam(model_lstm.parameters(),lr=3e-3)
for ep in range(10):
    model_lstm.train(); run=0;ntr=0
    for xb,yb in tr_dl:
        opt.zero_grad(); mu,sig=model_lstm(xb); loss=gaussian_nll(mu,sig,yb).mean(); loss.backward(); opt.step()
        run+=loss.item()*len(xb); ntr+=len(xb)
    # val
    model_lstm.eval(); tot=0;n=0
    with torch.no_grad():
        for xb,yb in va_dl:
            mu,sig=model_lstm(xb); l=gaussian_nll(mu,sig,yb).mean().item(); tot+=l*len(xb); n+=len(xb)
    print(f'Epoch {ep+1:02d} | train NLL={run/ntr:.4f} | val NLL={tot/n:.4f}')
# Test
model_lstm.eval(); P_mu=[];P_sig=[];Y=[]
with torch.no_grad():
    for xb,yb in te_dl:
        mu,sig=model_lstm(xb); P_mu.append(mu.numpy()); P_sig.append(sig.numpy()); Y.append(yb.numpy())
P_mu=np.vstack(P_mu); P_sig=np.vstack(P_sig); Y=np.vstack(Y)
nll_te=float(gaussian_nll(torch.tensor(P_mu),torch.tensor(P_sig),torch.tensor(Y)).mean())
crps_te=float(crps_gaussian(torch.tensor(P_mu),torch.tensor(P_sig),torch.tensor(Y)).mean())
mu_c, y_c = sc.inverse_transform(P_mu), sc.inverse_transform(Y)
rmse_mu=float(np.sqrt(((mu_c-y_c)**2).mean()))
print('LSTM te — NLL:',round(nll_te,3),'CRPS:',round(crps_te,3),'RMSE °C:',round(rmse_mu,3))

## 6) Calibration: PIT histogram & reliability for quantiles (CNN outputs)

In [ ]:
from math import erf
def pit_vals(mu,sigma,y):
    eps=1e-6; sigma=np.maximum(sigma,eps)
    z=(y-mu)/sigma
    return 0.5*(1+erf(z/np.sqrt(2.0)))
pit=pit_vals(P_mu[:,0],P_sig[:,0],Y[:,0])
plt.figure(figsize=(6,3)); plt.hist(pit,bins=15,edgecolor='k'); plt.title('PIT histogram (CNN)'); plt.xlabel('u'); plt.ylabel('count'); plt.show()
from scipy.stats import norm
qs=np.linspace(0.1,0.9,9); cov=[]
for q in qs:
    zq=norm.ppf(q); qhat=P_mu+zq*P_sig; cov.append(float((Y<=qhat).mean()))
plt.figure(figsize=(4,4)); plt.plot([0,1],[0,1],'k--'); plt.plot(qs,cov,marker='o'); plt.xlabel('Nominal'); plt.ylabel('Observed'); plt.title('Reliability (CNN)'); plt.grid(True); plt.show()

## 7) (Optional) Quantile head with pinball loss — starter code

In [ ]:
# See the slides for quantile regression motivation; a small example is included here.
class TinyCNNBase(nn.Module):
    def __init__(self):
        super().__init__(); self.conv1=nn.Conv1d(1,8,3,padding=1); self.conv2=nn.Conv1d(8,16,3,padding=1); self.pool=nn.AdaptiveAvgPool1d(1); self.out_dim=16
    def forward(self,x): x=x.transpose(1,2); h=torch.relu(self.conv1(x)); h=torch.relu(self.conv2(h)); h=self.pool(h).squeeze(-1); return h
class QuantileNet(nn.Module):
    def __init__(self, base, quantiles=(0.1,0.5,0.9)):
        super().__init__(); self.base=base; self.out=nn.Linear(base.out_dim,len(quantiles)); self.q=torch.tensor(quantiles,dtype=torch.float32).view(1,-1)
    def forward(self,x): h=self.base(x); return self.out(h)
def pinball_loss(q_pred,y,quantiles=(0.1,0.5,0.9)):
    qs=torch.tensor(quantiles,dtype=torch.float32,device=q_pred.device).view(1,-1)
    e=y-q_pred; return torch.mean(torch.maximum(qs*e,(qs-1)*e))
# Training loop omitted for brevity; see previous notebooks' pattern.